In [1]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB as MNB

In [2]:
tweet_df_location = pd.read_csv('../data/graphAnalysis/clean_climateTwitterData.csv')
tweet_df_location = tweet_df_location[tweet_df_location['text'].notna()]

In [3]:
import re
import string
from nltk.tokenize import TweetTokenizer

# This step was performed in the sentiment Analyzer
# def replace_urls(in_string, replacement=None):
#     # """Replace URLs in strings. See also: ``bit.ly/PyURLre``

#     # Args:
#     #     in_string (str): string to filter
#     #     replacement (str or None): replacment text. defaults to '<-URL->'

#     # Returns:
#     #     str
#     # """
#     replacement = '<-URL->' if replacement is None else replacement
#     pattern = re.compile('(https?://)?(\w*[.]\w+)+([/?=&]+\w+)*')
#     return re.sub(pattern, replacement, in_string)


def tokenize_only(in_string):
    """
    Convert `in_string` of text to a list of tokens using NLTK's TweetTokenizer
    """
    # reasonable, but adjustable tokenizer settings
    tokenizer = TweetTokenizer(preserve_case=False,
                               reduce_len=True,
                               strip_handles=False)
    tokens = tokenizer.tokenize(in_string)
    return tokens




In [4]:
# This step was performed in the sentiment Analyzer
# # remove urls and retweets and entities from the text
# tweet_df_location['text_clean'] = tweet_df_location['text'].apply(lambda row:replace_urls(row))

# #remove punctuations
# RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])  
# tweet_df_location['text_clean'] = tweet_df_location['text_clean'].str.replace(RE_PUNCTUATION, "")

In [5]:
from nltk.corpus import stopwords

# List of stopwords
stop_words= stopwords.words('english') #import stopwords from NLTK package
readInStopwords = pd.read_csv("pre_process/twitterStopWords.csv", encoding='ISO-8859-1') # import stopwords from CSV file as pandas data frame
readInStopwords = readInStopwords.wordList.tolist() # convert pandas data frame to a list
readInStopwords.append('http')
readInStopwords.append('https')

search_terms = ['#climateStrike','#climatestrike','#climatechange','#GreenNewDeal','#climatecrisis','#climateAction','#FridaysForFuture',
            '#environment','#globalwarming','#GlobalWarming','#ActOnClimate','#sustainability','#savetheplanet',
        '#bushfiresAustralia','#bushfires']
readInStopwords.extend(search_terms)
stop_list = stop_words + readInStopwords # combine two lists i.e. NLTK stop words and CSV stopwords
stop_list = list(set(stop_list)) # strore only unique values 
    
print(stop_list)

[' did', " i'm", " who'd", ' been', 'them', 'being', "haven't", 'd', " doesn't", ' do', 'your', 'hasn', 'you', 'ought', " they're", 'weren', ' and', 'her', ' my', "you're", 's', ' does', ' any', 'shouldn', "they're", ' twas', ' you', "isn't", ' am', 'wouldn', " you're", "she'd", 'doesn', 'into', " that's", ' which', 'yourself', ' most', 'has', 'didn', " there's", " mustn't", "wouldn't", 'i', 'an', 'm', ' on', 'this', 'these', 'some', " he'll", 'down', ' with', '#climatestrike', 'doing', "we'd", " he's", 'if', " you'll", 'because', 'did', 'myself', " mightn't", ' says', ' often', '#climateAction', ' then', 'again', "that's", " won't", " don't", " shan't", ' because', 'then', "mightn't", ' will', "won't", " she's", " where'll", ' own', 'under', 'out', ' this', 'nor', "don't", " would've", " what'd", "how's", "where's", " how's", ' across', ' who', ' of', '#climatechange', ' like', ' after', ' in', ' if', ' however', " ain't", 'between', 'and', ' likely', " didn't", ' wants', "couldn't", 

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = tweet_df_location['text_clean']
y = tweet_df_location['search_hashtags']


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.00, stop_words=stop_list, tokenizer=tokenize_only) # Use tf (raw term count) features for LDA.
X_Vect = tf_vectorizer.fit_transform(X)

c:\users\tasne\envs\newproject\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['#actonclimate', '#bushfiresaustralia', '#climateaction', '#fridaysforfuture', '#greennewdeal', "'", 'able', 'across', "ain't", 'almost', 'also', 'among', "can't", 'cannot', "could've", 'dear', 'either', 'else', 'ever', 'every', 'get', 'got', "how'd", "how'll", 'however', 'least', 'let', 'like', 'likely', 'may', 'might', "might've", 'must', "must've", 'neither', 'often', 'rather', 'said', 'say', 'says', 'since', 'tis', 'twas', 'us', 'wants', "what'd", "when'd", "when'll", "where'd", "where'll", "who'd", "who'll", "why'd", "why'll", "would've", 'yet'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_Vect, y, test_size=0.30, random_state=42, stratify=y)

In [10]:
X_train.shape

(50683, 87818)

In [11]:
y_train.shape

(50683,)

In [12]:
X_test.shape

(21722, 87818)

In [13]:
y_test.shape

(21722,)

In [14]:
mnb = MNB()

In [15]:
mnb.fit(X_train, y_train)  

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
predictions = mnb.predict(X_test)

In [17]:
from sklearn.metrics import confusion_matrix, classification_report

In [18]:
print(confusion_matrix(y_test,predictions))

[[   8    0    0  306    2  247    0    0    0    5    0    0]
 [   0    0    0  253    0   17    0    0    0    0    0    0]
 [   0    0  118  946    2  829    5    0    6    1    0    6]
 [   0    0    7 3913    2  898   15    0    6    6    0   10]
 [   0    0    7 1020   49  411    3    0    1    3    0    1]
 [   0    0    4  331    0 5162    4    2    0    0    0    4]
 [   0    0    0  916    0  236  220    0    4    0    0   35]
 [   0    0   10   83    0  772   12   33    0    0    0    1]
 [   0    0    0  888    1  164   22    0  170    0    0    1]
 [   0    0    0  730    0  404    0    0    0  243    0    0]
 [   0    0    0  224    0  153   16    0    8    0   15   14]
 [   0    0    2  967    0  188   15    0   11    0    0  554]]


In [19]:
print(classification_report(y_test,predictions))

c:\users\tasne\envs\newproject\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                   precision    recall  f1-score   support

    #actonclimate       1.00      0.01      0.03       568
       #bushfires       0.00      0.00      0.00       270
   #climateaction       0.80      0.06      0.11      1913
   #climatechange       0.37      0.81      0.51      4857
   #climatecrisis       0.88      0.03      0.06      1495
   #climatestrike       0.54      0.94      0.69      5507
     #environment       0.71      0.16      0.26      1411
#fridaysforfuture       0.94      0.04      0.07       911
   #globalwarming       0.83      0.14      0.23      1246
    #greennewdeal       0.94      0.18      0.30      1377
   #savetheplanet       1.00      0.03      0.07       430
  #sustainability       0.88      0.32      0.47      1737

         accuracy                           0.48     21722
        macro avg       0.74      0.23      0.23     21722
     weighted avg       0.66      0.48      0.39     21722



In [20]:
from sklearn import metrics

In [21]:
metrics.accuracy_score(y_test,predictions)

0.48269036000368293

- Accuracy is -- 0.4743117576650401 without Tokenization and Stopwords
- Accuracy is -- 0.4820458521314796 with Tokenization and Stopwords

In [22]:
strVal = ["Cuyahoga county eliminates use of plastic bags","Weather is still warm in winters and is not freezing"]
Value = tf_vectorizer.transform(strVal)
Value.shape

(2, 87818)

In [23]:
mnb.predict(Value)

array(['#environment', '#climatechange'], dtype='<U17')

In [24]:
from sklearn.model_selection import GridSearchCV
param_grid = {'alpha': [0.001, 0.01, 0.1, 1]}

grid = GridSearchCV(MNB(), param_grid, cv=10)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)

Best cross-validation score: 0.61
Best parameters:  {'alpha': 0.01}


- Best cross-validation score: 0.61
- Best parameters:  {'alpha': 0.01}

In [25]:
grid.predict(Value)

array(['#environment', '#globalwarming'], dtype='<U17')